<img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# LSEG Data Libraries for Python

**Data for Chains of Options**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial covers

* Retrieving Options Data &mdash; Single Maturity
* Retrieving Options Data &mdash; Multiple Maturities
* Selecting Sub Sets of the Options Data
* Visualization of Sub Sets of the Options Data

## Imports and Versions

The following imports several **packages** as used in the following.

In [1]:
import lseg.data as ld  # the LSEG Data Libraries for Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
cf.set_config_file(offline=True)  # set the plotting mode to offline

The following **Python and package versions** are used.

In [2]:
import sys
print(sys.version)

3.9.6 (default, Aug 11 2023, 19:44:49) 
[Clang 15.0.0 (clang-1500.0.40.1)]


In [4]:
ld.__version__

'2.0.0'

In [5]:
np.__version__

'1.23.4'

In [6]:
pd.__version__

'2.0.2'

In [7]:
cf.__version__

'0.17.3'

## Connecting to LSEG Data Libraries

In [8]:
ld.open_session()

<lseg.data.session.Definition object at 0x137727790 {name='workspace'}>

## Retrieving Options Data &mdash; Single Maturity

The `RIC` structure provides access to groups of `RICs` that have a common association &mdash; for example, the constituent members of the DAX 30 index or a list of options for a particular underlying contract, such as the DAX 30 index.

For example:

* `0#.GDAX` gives you a list of DAX 30 constituents 

* `0#GDAX*.EX` gives you a list of all options for all maturities for the DAX index
   
* `0#GDAXM5*.EX` gives you a list of all options for June 2025 maturity for the DAX index

### The Raw Data 

In what follows, data is retrieved that comprises fields for the **option type (put or call), the strike price, the closing price and the implied volatility**.

In [15]:
fields = ['PUTCALLIND', 'STRIKE_PRC', 'CF_CLOSE', 'IMP_VOLT']

Using the `ld.get_data()` method, allows data retrieval for a **single maturity of index options and multiple data fields** at the same time.

In [16]:
dax = ld.get_data('0#GDAXM5*.EX', fields=fields)

In [17]:
dax.head()

Instrument PUTCALLIND  STRIKE_PRC  CF_CLOSE  IMP_VOLT
0          .GDAXI       <NA>        <NA>  18557.27      <NA>
1  GDAX90000F5.EX       CALL        9000   10088.0   51.0017
2  GDAX90000R5.EX       PUT         9000      20.5   33.7916
3  GDAX92000F5.EX       CALL        9200    9896.5   49.7975
4  GDAX92000R5.EX       PUT         9200      22.1   33.2396

Let us pick and store the closing value for the index.

In [18]:
GDAXI = dax.iloc[0]['CF_CLOSE']

In [19]:
GDAXI

18557.27

To work with the data, the put options related data rows and call options related data rows are separated into two different `DataFrame` objects.

In [20]:
puts = dax[dax['PUTCALLIND'] == 'PUT ']
calls = dax[dax['PUTCALLIND'] == 'CALL']

In [21]:
puts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, 2 to 168
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  84 non-null     string 
 1   PUTCALLIND  84 non-null     string 
 2   STRIKE_PRC  84 non-null     Int64  
 3   CF_CLOSE    84 non-null     Float64
 4   IMP_VOLT    80 non-null     Float64
dtypes: Float64(2), Int64(1), string(2)
memory usage: 4.2 KB


In [22]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, 1 to 167
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  84 non-null     string 
 1   PUTCALLIND  84 non-null     string 
 2   STRIKE_PRC  84 non-null     Int64  
 3   CF_CLOSE    84 non-null     Float64
 4   IMP_VOLT    84 non-null     Float64
dtypes: Float64(2), Int64(1), string(2)
memory usage: 4.2 KB


It might make sense, to restrict the options data to those options that are not too far in or out of the money.

In [23]:
limit = 2000

In [24]:
puts = puts[abs(puts['STRIKE_PRC'] - GDAXI) < limit]
calls = calls[abs(calls['STRIKE_PRC'] - GDAXI) < limit]

In [31]:
puts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 68 to 142
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  38 non-null     string 
 1   PUTCALLIND  38 non-null     string 
 2   STRIKE_PRC  38 non-null     Int64  
 3   CF_CLOSE    38 non-null     Float64
 4   IMP_VOLT    38 non-null     Float64
dtypes: Float64(2), Int64(1), string(2)
memory usage: 1.9 KB


In [32]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 67 to 141
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  38 non-null     string 
 1   PUTCALLIND  38 non-null     string 
 2   STRIKE_PRC  38 non-null     Int64  
 3   CF_CLOSE    38 non-null     Float64
 4   IMP_VOLT    38 non-null     Float64
dtypes: Float64(2), Int64(1), string(2)
memory usage: 1.9 KB


### The Data Visualized

The separation allows for easily **visualization** of the closing prices and the implied volatilities.

In [41]:
puts.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(subplots=True,
                                                            mode='lines+markers',
                                                            size=6)

In [42]:
calls.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(subplots=True,
                                                             mode='lines+markers',
                                                             size=6)

## Retrieving Data &mdash; Multiple Maturities

Via the `LSEG Data Libraries`, options data can be retrieved also for all maturities of an options chain at once.

### The Raw Data

The maturity date data field (`EXPIR_DATE`) is added to distinguish the maturity dates.

In [43]:
fields = ['PUTCALLIND', 'EXPIR_DATE', 'STRIKE_PRC', 'CF_CLOSE', 'IMP_VOLT']

In [44]:
dax = ld.get_data('0#GDAX*.EX', fields=fields)

In [45]:
dax.head()

Instrument PUTCALLIND EXPIR_DATE  STRIKE_PRC  CF_CLOSE  IMP_VOLT
0          .GDAXI       <NA>        NaT        <NA>  18557.27      <NA>
1  GDAX60000F4.EX       CALL 2024-06-21        6000   12564.1  183.8289
2  GDAX60000R4.EX       PUT  2024-06-21        6000       0.1    154.47
3  GDAX61000F4.EX       CALL 2024-06-21        6100   12464.3  180.7053
4  GDAX61000R4.EX       PUT  2024-06-21        6100       0.1  152.2699

Again, put and call option data is separated.

In [46]:
puts = dax[dax['PUTCALLIND'] == 'PUT ']
calls = dax[dax['PUTCALLIND'] == 'CALL']

Again, options too far in or out of the money are removed.

In [47]:
puts = puts[abs(puts['STRIKE_PRC'] - GDAXI) < limit]
calls = calls[abs(calls['STRIKE_PRC'] - GDAXI) < limit]

In [48]:
puts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 539 entries, 132 to 2234
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Instrument  539 non-null    string        
 1   PUTCALLIND  539 non-null    string        
 2   EXPIR_DATE  539 non-null    datetime64[ns]
 3   STRIKE_PRC  539 non-null    Int64         
 4   CF_CLOSE    539 non-null    Float64       
 5   IMP_VOLT    539 non-null    Float64       
dtypes: Float64(2), Int64(1), datetime64[ns](1), string(2)
memory usage: 31.1 KB


In [49]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 539 entries, 131 to 2233
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Instrument  539 non-null    string        
 1   PUTCALLIND  539 non-null    string        
 2   EXPIR_DATE  539 non-null    datetime64[ns]
 3   STRIKE_PRC  539 non-null    Int64         
 4   CF_CLOSE    539 non-null    Float64       
 5   IMP_VOLT    539 non-null    Float64       
dtypes: Float64(2), Int64(1), datetime64[ns](1), string(2)
memory usage: 31.1 KB


From all option maturites ...

In [50]:
exp_dates = sorted(list(set(puts['EXPIR_DATE'])))
exp_dates

[Timestamp('2024-06-21 00:00:00'),
 Timestamp('2024-07-19 00:00:00'),
 Timestamp('2024-08-16 00:00:00'),
 Timestamp('2024-09-20 00:00:00'),
 Timestamp('2024-12-20 00:00:00'),
 Timestamp('2025-03-21 00:00:00'),
 Timestamp('2025-06-20 00:00:00'),
 Timestamp('2025-09-19 00:00:00'),
 Timestamp('2025-12-19 00:00:00'),
 Timestamp('2026-03-20 00:00:00'),
 Timestamp('2026-06-19 00:00:00'),
 Timestamp('2026-09-18 00:00:00'),
 Timestamp('2026-12-18 00:00:00'),
 Timestamp('2027-03-19 00:00:00'),
 Timestamp('2027-12-17 00:00:00'),
 Timestamp('2028-12-15 00:00:00')]

... only a sub-set selected.

In [51]:
exp_dates = exp_dates[0:7:3]

In [52]:
exp_dates

[Timestamp('2024-06-21 00:00:00'),
 Timestamp('2024-09-20 00:00:00'),
 Timestamp('2025-06-20 00:00:00')]

### The Data Visualized

The following visualizes **closing prices and implied volatilities in a single plot per maturity**. First, for the **puts**.

In [64]:
for expiry in exp_dates:
    to_plot = puts[puts['EXPIR_DATE'] == expiry]
    to_plot.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(
                                                                    title=expiry.strftime('%Y-%m-%d'),
                                                                    secondary_y='IMP_VOLT',
                                                                    mode='lines+markers',
                                                                    size=6,
                                                                    xTitle='strike price')

Second, for the **calls**.

In [65]:
for expiry in exp_dates:
    to_plot = calls[calls['EXPIR_DATE'] == expiry]
    to_plot.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(title=expiry.strftime('%Y-%m-%d'),
                                                                   secondary_y='IMP_VOLT',
                                                                   mode='lines+markers',
                                                                size=6,
                                                                   xTitle='strike price')

## Comparing Prices & Implied Volatilities

However, often one wishes to compare prices and implied volatilites, respectively, for **different maturities in a single plot**.

### Puts

The code below collects **put option closing prices and implied volatilities in different columns** of two separate `DataFrame` objects.

In [66]:
puts_close = pd.DataFrame()
puts_vols = pd.DataFrame()
for expiry in exp_dates:
    sub_set = puts[puts['EXPIR_DATE'] == expiry]
    puts_close[expiry] = sub_set.set_index('STRIKE_PRC')['CF_CLOSE']
    puts_vols[expiry] = sub_set.set_index('STRIKE_PRC')['IMP_VOLT']

Now, closing prices and implied volatilities can be compared easily in a single plot.

In [83]:
puts_close.dropna(inplace=True)
puts_vols.dropna(inplace=True)

In [84]:
puts_close.dropna().iplot(mode='markers', symbol=['circle-dot', 'cross', 'diamond'], size=6, xTitle='strike price')

In [85]:
puts_vols.iplot(mode='markers', symbol=['circle-dot', 'cross', 'diamond'], size=6, xTitle='strike price')

### Calls

This code does the same for the **call options**.

In [87]:
calls_close = pd.DataFrame()
calls_vols = pd.DataFrame()
for expiry in exp_dates:
    sub_set = calls[calls['EXPIR_DATE'] == expiry]
    calls_close[expiry] = sub_set.set_index('STRIKE_PRC')['CF_CLOSE']
    calls_vols[expiry] = sub_set.set_index('STRIKE_PRC')['IMP_VOLT']

In [88]:
calls_close.dropna(inplace=True)
calls_vols.dropna(inplace=True)

In [89]:
calls_close.iplot(mode='markers', symbol=['circle-dot', 'cross', 'diamond'], size=6)

In [90]:
calls_vols.iplot(mode='markers', symbol=['circle-dot', 'cross', 'diamond'], size=6)

## Conclusions

This tutorial covers:

* Retrieving Options Data &mdash; Single Maturity
* Retrieving Options Data &mdash; Multiple Maturities
* Selecting Sub Sets of the Options Data
* Visualization of Sub Sets of the Options Data

## LSEG Data Libraries Developer Resources

* [API Pages](https://developers.lseg.com/en/api-catalog) 
* [Q&A Forums](https://community.developers.refinitiv.com/index.html) 

Data Item Browser Application: Type `DIB` into Workspace Search Bar.

* [Article on Chains](https://developers.lseg.com/en/article-catalog/article/simple-chain-objects-ema-part-1)

<img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">